# Master class
* 최상위 클래스: 아래의 모든 클래스는 이 클래스를 상속받아야 한다
* 코드를 작성하는 데 있어서 사용자가 기본적으로 지켜야 할 원칙 등을 정의한다

In [8]:
class Master:
    def __init__(self):
        pass
    
    @classmethod
    def inherited_from(cls, parent_cls):
        if parent_cls in cls.__mro__:
            return True
        else:
            raise cls.NotInheritedError()
            
    class NotInheritedError(Exception):
        pass

# Data class
* 데이터 소스에 접근하는 wrapper 클래스
* 실제 데이터는 RESTful api (http://robofin.pythonanywhere.com/api/) 로 제공한다

* Sub classes (유형)
    * MarketData: 시장데이터 (시고종저, 거래량 등)
    * FinancialData: 재무데이터 
    * AlternativeData: ...
    * NewsData: ...

* 사용법: 한개 이상의 sub class를 상속하거나, 완성된 Data 클래스 자체를 재상속

In [37]:
import requests

class Data(Master):
    def __init__(self):
        pass
    
    @property
    def _data_type(self):
        return None
    
    
class MarketData(Data):
    def price(self, asof=None, start=None, end=None, tickers=None):
        return self.get_marketdata(asof=asof, start=start, end=end, tickers=tickers, item='price')
    
    def get_marketdata(self, asof=None, start=None, end=None, tickers=None, item=None):
        raise NotImplementedError
    
    
class FinancialData(Data):
    def get_fisyear(self, asof=None, ticker=None):
        return None
    
    def get_financialdata(self, asof=None, start=None, end=None, tickers=None, item=None):
        raise NotImplementedError
        
    
class AlternativeData(Data):
    pass
    
    
class KoreaStockData(MarketData, FinancialData):
    def __init__(self):
        pass

### api 사용예

In [40]:
url = 'http://robofin.pythonanywhere.com/api/marketdata?start=2016-12-31&end=2017-12-31&ticker=A005930&item=price'
requests.get(url).json()['results']

[{'ticker': 'A005930', 'date': '2016-12-31', 'price': 1802000.0},
 {'ticker': 'A005930', 'date': '2017-01-31', 'price': 1973000.0},
 {'ticker': 'A005930', 'date': '2017-02-28', 'price': 1922000.0},
 {'ticker': 'A005930', 'date': '2017-03-31', 'price': 2060000.0},
 {'ticker': 'A005930', 'date': '2017-04-30', 'price': 2231000.0},
 {'ticker': 'A005930', 'date': '2017-05-31', 'price': 2235000.0},
 {'ticker': 'A005930', 'date': '2017-06-30', 'price': 2377000.0},
 {'ticker': 'A005930', 'date': '2017-07-31', 'price': 2410000.0},
 {'ticker': 'A005930', 'date': '2017-08-31', 'price': 2316000.0},
 {'ticker': 'A005930', 'date': '2017-09-30', 'price': 2564000.0},
 {'ticker': 'A005930', 'date': '2017-10-31', 'price': 2754000.0},
 {'ticker': 'A005930', 'date': '2017-11-30', 'price': 2540000.0},
 {'ticker': 'A005930', 'date': '2017-12-31', 'price': 2548000.0}]

# Strategy class
* 투자전략을 설정하는 클래스
* 제공된 데이터의 유형에 따라 Strategy 클래스의 유형이 달라짐
* 필요한 경우 bm 포지션도 생성

In [41]:
class Strategy(Master):
    def __init__(self, data=None):
        self.data = data
        self.type = data._data_type
        
    def selection(self):
        raise NotImplementedError
    
    def selection_bm(self):
        pass
    
    def weighting(self):
        pass

# Exit class
* 포지션의 일부 또는 전부를 exit 하는 전략을 구현하는 클래스
* sub classes
    * ProfitTake
    * Losscut
    * Reentry
* 사용법: 하나 이상의 sub classes를 상속하거나, DefaultExit를 상속

In [33]:
class Exit(Master):
    def __init__(self, data=None):
        self.data = data
        self.type = data._data_type

class ProfitTake(Exit):
    def __init__(self):
        pass
    
class Losscut(Exit):
    def __init__(self):
        pass
    
class Reentry(Exit):
    def __init__(self):
        pass
    
class DefaultExit(ProfitTake, Losscut, Reentry):
    def __init__(self):
        pass

# Review class
* 성과측정을 구현한 class

In [13]:
class Review(Master):
    def __init__(self):
        pass
    
    def get_annualized_return(self):
        return None
    
    def get_volatility(self):
        return None
    
    def get_sharpe(self):
        return None
    
    def get_stats(self):
        return None

# Backtester class
* 100% 우리가 관리하는 클래스
* 이 클래스를 상속(확장)할 수 있게 열어두나...? 생각해보자

In [20]:
class Backtester:
    def __init__(self, strategy=None, data=None, exit=DefaultExit, review=Review):
        self.strategy = self._modulize(strategy, Strategy)
        self.data = self._modulize(data, Data)
        self.exit = self._modulize(exit, Exit)
        self.review = self._modulize(review, Review)
        
    def _modulize(self, module, parent_cls):
        if module.inherited_from(parent_cls):
            return module
        
    def run(self):
        pass

# Use case

In [34]:
class MyStrat(Strategy):
    def __init__(self, data=KoreaStockData):
        pass
    
class MyExit(DefaultExit):
    def __init__(self, data=KoreaStockData):
        pass
    
class MyReview(Review):
    def __init__(self):
        pass

In [35]:
bt = Backtester(strategy=MyStrat, data=KoreaStockData, exit=MyExit, review=MyReview)